In [6]:
%matplotlib inline
# Importam biblioteci utile pentru manipularea datelor si AI
import pandas as pd   # pentru a lucra cu tabele (date structurate)
import numpy as np    # pentru calcule matematice

import os

import torch
import torch.nn as nn
from torch.autograd import Variable

from tensorflow.keras.preprocessing.text import Tokenizer   # transforma textul in numere
from keras.preprocessing.sequence import pad_sequences      # face ca toate textele sa aiba aceeasi lungime

# Biblioteci pentru procesarea limbajului natural (text)
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')  # descarcă lista de cuvinte comune
nltk.download('wordnet')
stop_words = stopwords.words('german')   # folosim lista de cuvinte comune in germana (care nu ajuta modelul)
import re
pd.set_option('display.precision',20)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\const\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\const\AppData\Roaming\nltk_data...


In [7]:
# ------------------------------------------------------------
# Incarcam datele (texte de antrenament, test si validare)
# ------------------------------------------------------------
# training.txt contine exemple folosite pentru invatarea AI-ului
df_train = pd.read_csv('dataset/training.txt', names=['id', 'lat','long','tweet'], header=None)
df_train.head()

,id,lat,long,tweet
0,119165,51.81006711409395393275,10.19133093525180200345,"Seit d Vase: ""Wenn ich kaputt gang, bringt das..."
1,100377,51.91818791946306532736,10.59924460431654580361,Haha bin au w isch der amig au so richtig lang...
2,109550,52.71107382550334818916,9.98737410071942477430,isch d hiltl dachterrasse amne samstig viel bs...
3,111440,52.38671140939595716191,11.70061151079136863018,Ich fühle mich wie die Weimarer Republik... .....
4,116670,52.31463087248320675826,9.70183453237410375891,Eui liebschte Lunchidee zum Mitneh? 😬 En Grill...


In [8]:
# test.txt contine exemple noi pe care le vom folosi pentru a verifica modelul
df_test = pd.read_csv('dataset/test.txt', names=['id','tweet'], header=None)
df_test.head()

,id,tweet
0,300121,👩min vibi funktionkert nöd... 👧hesch d'batteri...
1,302441,Ich: Also langsam söti scho schlafe Au Ich: He...
2,300266,Hez hie ou lüt wo dr ganz tag ine biudschirm m...
3,300911,je neui wohnig 😎 neua job 😎 eigeni kuchi 😎 abe...
4,302681,Schön wies grad chunt cho hagle u du nid d müg...


In [9]:
# validation.txt este folosit pentru a evalua modelul in timpul antrenarii
df_valid = pd.read_csv('dataset/validation.txt', names=['id', 'lat','long','tweet'], header=None)
df_valid.head()

,id,lat,long,tweet
0,203001,52.09838926174496265276,10.35449640287770023406,wenn mer anere Party bi Kollege en neue Bro fi...
1,200313,51.62986577181206371279,7.72345323741007305784,Heii guetä wuchestart gha? Jo wunderbar und du...
2,201966,52.24255033557045635462,8.96758992805755283939,Wieso ned? De werds eim emel ned langwiilig. O...
3,201123,52.24255033557045635462,8.96758992805755283939,Hani welle vo zueri uf lausanne denn hetts e d...
4,200374,51.70194630872484253814,8.41690647482014497882,Mir isch die Applikation plagiert worde as Dis...


In [10]:
# ------------------------------------------------------------
# Functie care curata textul (elimina caractere inutile)
# ------------------------------------------------------------
def clean_text(input_text):
    text = re.sub(r'\W',' ', input_text) # Elimină toate caracterele non-cuvinte
    text = re.sub(r'\d+',' ', text) # Elimină toate cifrele
    text = input_text.lower() # Convertim textul în litere mici
    text = re.sub(r'\s+[a-z]\s+',' ', text) # Elimină toate literele singure
    text = re.sub(r'^\s+','', text) # Elimină spațiile de la începutul textului
    text = re.sub(r'\s+$','', text) # Elimină spațiile de la sfârșitul textului
    text = re.sub(r'\s+',' ', text) # Elimină toate spațiile multiple
    text = text.split(' ') # Împarte cuvintele în tokeni
    text = [word for word in text if word not in stop_words] # Elimină cuvintele comune
    text = [WordNetLemmatizer().lemmatize(word) for word in text] # Aduce cuvintele la forma de bază
    text = ' '.join(text)

    return text

In [11]:
df_train['tweet'] = df_train.tweet.apply(lambda x: clean_text(x))

In [12]:
df_test['tweet'] = df_test.tweet.apply(lambda x: clean_text(x))

In [13]:
df_valid['tweet'] = df_valid.tweet.apply(lambda x: clean_text(x))

In [14]:
# Pregătirea datelor pentru model
X = df_train['tweet']  # Textul de antrenament (caracteristicile)
y_lat = df_train['lat']  # Latitudinea țintă (normalizată)
y_long = df_train['long']  # Longitudinea țintă (normalizată)

X_valid = df_valid['tweet']  # Date de validare
y_lat_valid = df_valid['lat']
y_long_valid = df_valid['long']

test = df_test['tweet']  # Date de test

In [15]:
# Crearea tokenizer-ului (converteste cuvintele în numere)
num_words = 2000  # Vom lua în considerare doar primele 2000 de cuvinte cele mai frecvente
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(df_train['tweet'].values)  # Învață vocabularul din datele de antrenament

In [16]:
# Transformarea textului în secvențe numerice și padding
X = tokenizer.texts_to_sequences(df_train['tweet'].values)  # Converteste textul în numere
X = pad_sequences(X, maxlen=num_words)  # Face ca toate secvențele să aibă aceeași lungime

X_valid = tokenizer.texts_to_sequences(df_valid['tweet'].values)
X_valid = pad_sequences(X_valid, maxlen=num_words)

test = tokenizer.texts_to_sequences(df_test['tweet'].values)
test = pad_sequences(test, maxlen=num_words)

In [21]:
class Net(nn.Module):
  def __init__(self,num_words,out_count):
    super(Net,self).__init__()
    
    # ------------------------------------------------------------
    # ARHITECTURA PENTRU PREDICȚIA LATITUDINII
    # ------------------------------------------------------------
    
    # Strat linear 1 pentru latitudine: transformă de la num_words la 1000 de neuroni
    self.lin1_lat=nn.Linear(num_words,1000)
    
    # Strat linear 2: reducere de la 1000 la 500 de neuroni
    self.lin2_lat=nn.Linear(1000,500)
    
    # Strat linear 3: reducere de la 500 la 50 de neuroni (bottleneck layer)
    self.lin3_lat=nn.Linear(500,50)
    
    # Strat linear 4: expansiune de la 50 la 200 de neuroni
    self.lin4_lat=nn.Linear(50,200)
    
    # Strat linear de output pentru latitudine: de la 200 la out_count neuroni
    self.lin5_lat=nn.Linear(200,out_count)
    
    # ------------------------------------------------------------
    # ARHITECTURA PENTRU PREDICȚIA LONGITUDINII
    # ------------------------------------------------------------
    
    # Aceeași arhitectură ca pentru latitudine, dar cu ponderi separate
    self.lin1_long=nn.Linear(num_words,1000)
    self.lin2_long=nn.Linear(1000,500)
    self.lin3_long=nn.Linear(500,50)
    self.lin4_long=nn.Linear(50,200)
    self.lin5_long=nn.Linear(200,out_count)

  def forward(self,X):
    # ------------------------------------------------------------
    # PROPAGARE ÎN AINATE PENTRU LATITUDINE
    # ------------------------------------------------------------
    
    # Trecem input-ul prin toate straturile pentru latitudine
    lin1_lat = self.lin1_lat(X)        # num_words -> 1000
    lin2_lat = self.lin2_lat(lin1_lat) # 1000 -> 500
    lin3_lat = self.lin3_lat(lin2_lat) # 500 -> 50 (bottleneck)
    lin4_lat = self.lin4_lat(lin3_lat) # 50 -> 200
    lin5_lat = self.lin5_lat(lin4_lat) # 200 -> out_count (output final)
    
    # ------------------------------------------------------------
    # PROPAGARE ÎN AINATE PENTRU LONGITUDINE
    # ------------------------------------------------------------
    
    # Trecem input-ul prin toate straturile pentru longitudine
    lin1_long = self.lin1_long(X)        # num_words -> 1000
    lin2_long = self.lin2_long(lin1_long) # 1000 -> 500
    lin3_long = self.lin3_long(lin2_long) # 500 -> 50 (bottleneck)
    lin4_long = self.lin4_long(lin3_long) # 50 -> 200
    lin5_long = self.lin5_long(lin4_long) # 200 -> out_count (output final)
    return lin5_lat,lin5_long

In [22]:
# Convertirea datelor de antrenament în tensori PyTorch și transferul pe GPU
x_train = Variable(torch.FloatTensor(X)).to('cuda')  # Convertim datele de intrare de antrenament în tensor și le mutăm pe GPU
x_valid = Variable(torch.FloatTensor(X_valid)).to('cuda')  # Convertim datele de validare în tensor și le mutăm pe GPU

# Pregătirea țintelor (targets) pentru latitudine - antrenament
y_train_lat = Variable(torch.FloatTensor(y_lat)).to('cuda')  # Convertim latitudinile de antrenament în tensor
y_train_lat = y_train_lat / 100  # Normalizăm valorile latitudinii (împărțim la 100)

# Pregătirea țintelor (targets) pentru longitudine - antrenament
y_train_long = Variable(torch.FloatTensor(y_long)).to('cuda')  # Convertim longitudinile de antrenament în tensor
y_train_long = y_train_long / 100  # Normalizăm valorile longitudinii (împărțim la 100)

# Pregătirea țintelor (targets) pentru latitudine - validare
y_valid_lat = Variable(torch.FloatTensor(y_lat_valid)).to('cuda')  # Convertim latitudinile de validare în tensor
y_valid_lat = y_valid_lat / 100  # Normalizăm valorile latitudinii (împărțim la 100)

# Pregătirea țintelor (targets) pentru longitudine - validare
y_valid_long = Variable(torch.FloatTensor(y_long_valid)).to('cuda')  # Convertim longitudinile de validare în tensor
y_valid_long = y_valid_long / 100  # Normalizăm valorile longitudinii (împărțim la 100)

In [23]:
# Inițializarea modelului de rețea neuronală
model = Net(num_words, 1).to('cuda')  # Creăm instanța modelului și o mutăm pe GPU

# Definirea funcției de loss (funcția de cost)
loss_fn = torch.nn.L1Loss().to('cuda')  # Mean Absolute Error (MAE) - mutată pe GPU

# Configurarea optimizerului (algoritmul de optimizare)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)  # Stochastic Gradient Descent

In [ ]:
# Bucla de antrenament a modelului pentru 2000 de epoci (iterații)
for epoch in range(0, 2000):
  
  # Resetarea gradientelor de la iterația anterioară
  optimizer.zero_grad()  # Este CRITIC să resetăm gradientii înainte de fiecare pas
  
  # Forward pass: propagarea datelor prin model pentru a obține predicții
  out_lat, out_long = model(x_train)  # Modelul returnează două output-uri (latitudine și longitudine)
  
  # Calculul funcției de loss pentru latitudine
  # Reshape este necesar pentru a asigura forma corectă (batch_size, ) vs (batch_size, 1)
  loss_lat = loss_fn(out_lat.reshape(out_lat.shape[0]), y_train_lat)
  
  # Calculul funcției de loss pentru longitudine
  loss_long = loss_fn(out_long.reshape(out_long.shape[0]), y_train_long)
  
  # Loss total: sumă a celor două loss-uri individuale
  loss = loss_lat + loss_long  # Combinăm loss-urile pentru ambele output-uri
  
  # Backward pass: calculul gradientelor prin backpropagation
  loss.backward()  # Calculează derivatele (gradienții) pentru toți parametrii
  
  # Actualizarea parametrilor modelului folosind optimizer
  optimizer.step()  # Ajustează ponderile pe baza gradientelor calculate
  
  # Afișarea progresului antrenării
  print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 5.487862586975098
epoch 1, loss 4.428788661956787
epoch 2, loss 3.8716180324554443
epoch 3, loss 3.4950337409973145
epoch 4, loss 3.2356629371643066
epoch 5, loss 3.130373477935791
epoch 6, loss 3.1018710136413574
epoch 7, loss 3.071549415588379
epoch 8, loss 3.0220491886138916
epoch 9, loss 2.876884698867798
epoch 10, loss 2.8862481117248535
epoch 11, loss 3.000967025756836
epoch 12, loss 3.8683767318725586
epoch 13, loss 3.8403830528259277
epoch 14, loss 3.9151859283447266
epoch 15, loss 3.784618854522705
epoch 16, loss 3.885629177093506
epoch 17, loss 3.7281665802001953
epoch 18, loss 3.8532965183258057
epoch 19, loss 3.677177667617798
epoch 20, loss 3.8176400661468506
epoch 21, loss 3.6076159477233887
epoch 22, loss 3.774136543273926
epoch 23, loss 3.538802146911621
epoch 24, loss 3.727142333984375
epoch 25, loss 3.46859073638916
epoch 26, loss 3.6828088760375977
epoch 27, loss 3.4021174907684326
epoch 28, loss 3.641218662261963
epoch 29, loss 3.3287200927734375
epoch

In [ ]:
out_lat,out_long=model(x_valid)
loss_lat = loss_fn(out_lat.reshape(out_lat.shape[0]),y_valid_lat)
loss_long = loss_fn(out_long.reshape(out_long.shape[0]),y_valid_long)
loss=loss_lat+loss_long
print('loss {}'.format(loss.item()))

In [ ]:
x_test=Variable(torch.FloatTensor(test)).to('cuda')
out_lat,out_long=model(x_test)

In [ ]:
df_test['lat']=out_lat.cpu().detach().numpy()*100
df_test['long']=out_long.cpu().detach().numpy()*100

In [ ]:
df_test=df_test.drop('tweet', axis=1)

In [ ]:
df_test

In [ ]:
df_test.to_csv('dataset/rezultatPytorch2000.txt', index=False, decimal='.', sep=',', float_format='%.20f')